In [6]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import datetime
from lxml import html
import os 
import csv
from random import randint

On récupère les choix utilisatuer

In [27]:
print("\n \t $$$$$$$$$$$$$$\n\n      $$$  FINANCE APP     §§§\n \n \t $$$$$$$$$$$$$$\n \n ")


 	 $$$$$$$$$$$$$$

      $$$  FINANCE APP     §§§
 
 	 $$$$$$$$$$$$$$
 
 


In [28]:
choix = input("\n 1 - SOPRA\n 2 - L'OREAL\n 3 - AIR LIQUIDE\n 4 - BIO-MERIEUX\n 5 - TECHNIP\n 6 - ATOS\n 7 - Quitter\n \n  Choisissez un titre: ")


 1 - SOPRA
 2 - L'OREAL
 3 - AIR LIQUIDE
 4 - BIO-MERIEUX
 5 - TECHNIP
 6 - ATOS
 7 - Quitter
 
  Choisissez un titre: 2


On choisi un titre en fonction du choix de l'utilisateur

In [29]:
if choix == '1':
    titre = "SOP.PA"
    url = "https://www.boursorama.com/cours/1rPSOP/"
    action = "sopra"
elif choix == '2':
    titre = "OR.PA"
    url = "https://www.boursorama.com/cours/1rPOR/"
    action = "loreal"
elif choix == '3':
    titre = "AI.PA"
    url = "https://www.boursorama.com/cours/1rPAI/"
    action = "airliquide"
elif choix == '4':
    titre = "BIM.PA"
    url = "https://www.boursorama.com/cours/1rPBIM/"
    action = "biomerieux"
elif choix == '5':
    titre = "FTI.PA"
    url = "https://www.boursorama.com/cours/1rPAI/"
    action = "technipfmc"
elif choix == '6':
    titre = "ATO.PA"
    url = "https://www.boursorama.com/cours/1rPATO/"
    action = "atos"
    
elif choix == '7':
    exit()
else:
    print("Le titre choisi n'est pas pris en compte. \n ")
    exit()
    

print(titre)

OR.PA


Fonction pour changer le format des dates de AAAA-MM-JJ HH:MM:SS -> AAAA-MM-JJ HH:MM

In [30]:
def reformat_date(index):
    date = datetime.datetime.strptime(index, '%Y-%m-%d %H:%M:%S')
    date = datetime.datetime.strftime(date, '%Y-%m-%d %H:%M')
    
    return date

On définit la fonction qui chargera les données pour un indicateur technique

In [31]:
def analyse(titre ,fonction):
    periode = "20"
    
    #Les différentes parties de l'url
    part1_url_analyse = "https://www.alphavantage.co/query?function="
    part2_url_analyse = "&symbol="
    part3_url_analyse = "&interval=1min&time_period="
    part4_url_analyse = "&series_type=close&apikey=2FWTNAS6MFKAV1XH"
    
    #On construit l'url
    url_analyse = part1_url_analyse + fonction + part2_url_analyse + titre + part3_url_analyse + periode + part4_url_analyse
    
    #On exécute la requete
    response_an = requests.get(url_analyse)
    
    #On convertit le résultat de la requete en json puis en dataFrame
    data_an = json.loads(response_an.content)
    analyses = pd.DataFrame.from_dict(data_an["Technical Analysis: "+fonction], orient='index')
    
    return analyses
    

In [32]:
titre

'OR.PA'

FONTION POUR OBTENIR LES INDICATEURS TECHNIQUES

In [33]:
def analyse_tech(titre):
    #On construit l'url de la requete API avec une concaténation de chaines de caractères
    
    #Mais avant on prépare les parties de l'url
    part1_URL = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol="
    part2_URL = "&interval=1min&outputsize=full&apikey=2FWTNAS6MFKAV1XH"
    
    #Maintenant on construit l'url pour l'action en bourse 
    url = part1_URL + titre + part2_URL
    
    #On exécute la requête
    response = requests.get(url)
    
    #On convertit la réponse au format json
    data = json.loads(response.content)
    
    #On convertit en dataFrame
    df_data = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index')
    
    #On change les index des dataframe au format AAAA-MM-JJ HH:MM
    df_data.index = list(map(reformat_date, df_data.index))
    
    #Liste qui va contenir les fonctions à analyser
    fonctions = ['BBANDS', 'RSI', 'MACD', 'STOCH']
    
    #On calcule tous les indicateurs techniques
    for fonction in fonctions:
        analyse_tech = analyse(titre, fonction)
    
        #on joint le resultat au dataframe
        df_data = pd.concat([df_data, analyse_tech], axis=1, join_axes=[analyse_tech.index])
        
    return df_data

On appelle la fonction pour l'analyse technique

In [34]:
try:
    df_data = analyse_tech(titre)
except:
    print("\n La connexion semble lente, veuillez vérifier votre connexion internet :) ! \n")
    exit()

Fontion pour obtenir les indicateurs financiers

In [10]:
def analyse_fin(url):
    
    analyse_fin = []
    
    #recupere le PER
    page = requests.get(url)
    tree = html.fromstring(page.content)
    per = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[4]/td[2]/text()')
    s= per[0].split('        ')
    b = s[2][0:-1]
    analyse_fin.append(float(b))
    
    #Repcupere le dividende par action
    div = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[1]/td[4]/text()')
    div = div[0].split('        ')
    div = div[2][0:-1]
    analyse_fin.append(float(div))
    
    #recupere le BNA
    bna = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[3]/td[3]/text()')
    bna = bna[0].split('        ')
    bna = bna[2][0:-1]
    analyse_fin.append(float(bna))
    
    #recupere le rendement
    rd = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[2]/td[3]/text()')
    rd = rd[0].split('        ')
    rd = rd[2][0:-2]
    analyse_fin.append(float(rd))
    
    return analyse_fin
    
    
    
    
    
    
    

On charge les indicateurs financiers dans une liste dans cet ordre : -------
    1- PER , 
    2- Dividende , 
    3- BNA , 
    4- Rendement , 

In [11]:
finance = analyse_fin(url)

In [12]:
finance

[30.48, 0.55, 2.32, 0.78]

On supprime les NA

In [13]:
df_data = df_data.dropna()

In [14]:
df_data

,1. open,2. high,3. low,4. close,5. volume,Real Middle Band,Real Lower Band,Real Upper Band,RSI,MACD_Signal,MACD,MACD_Hist,SlowK,SlowD
2019-02-07 05:45,63.9000,63.9000,63.9000,63.9000,156,64.1250,63.8576,64.3924,53.4977,0.1377,0.0497,-0.0880,0.0000,0.0000
2019-02-07 05:50,64.0000,64.0000,64.0000,64.0000,685,64.1200,63.8472,64.3928,56.6818,0.1185,0.0419,-0.0767,33.3333,11.1111
2019-02-07 05:54,64.2000,64.2000,64.2000,64.2000,250,64.1250,63.8502,64.3998,62.1394,0.1051,0.0512,-0.0539,66.6667,33.3333
2019-02-07 06:03,64.0000,64.2000,64.0000,64.2000,76,64.1250,63.8502,64.3998,62.1394,0.0957,0.0579,-0.0377,100.0000,66.6667
2019-02-07 06:26,64.2000,64.2000,64.2000,64.2000,57,64.1250,63.8502,64.3998,62.1394,0.0890,0.0626,-0.0265,100.0000,88.8889
2019-02-07 06:28,64.2000,64.2000,64.2000,64.2000,310,64.1250,63.8502,64.3998,62.1394,0.0843,0.0655,-0.0189,100.0000,100.0000
2019-02-07 06:42,64.3000,64.3000,64.3000,64.3000,74,64.1350,63.8502,64.4198,64.8576,0.0825,0.0750,-0.0075,100.0000,100.0000
2019-02-07 06:47,64.3000,64.4000,64.3000,64.4000,731,64.1500,63.8434,64.4566,67.3266,0.0839,0.0895,0.0057,100.0000,100.0000
2019-02-07 06:50,64.5000,64.5000,64.5000,64.5000,65,64.1700,63.8288,64.5112,69.5766,0.0887,0.1079,0.0192,100.0000,100.0000
2019-02-07 06:56,64.5000,64.5000,64.5000,64.5000,271,64.1850,63.8147,64.5553,69.5766,0.0952,0.1211,0.0259,100.0000,100.0000


CALCUL DE LA NOTE DE L'ANALYSE TECHNIQUE ( sur 10 )

Fonction pour la note du RSI

In [15]:
def calcul_rsi(rsi):
    if( rsi < 30 ):
        note_rsi = 10 
    elif( 30 <= rsi <= 70 ):
        note_rsi = 5
    else:
        note_rsi = 0
        
    return note_rsi

Fonction pour la note du macd

In [16]:
def calcul_macd(macd):
    
    if (macd > 0):
        note_macd = 10
    elif(macd < 0):
        note_macd = 0
    else:
        note_macd = 5
        
    return note_macd

Fonction pour la note du stockastique

In [17]:
def calcul_stoch(stoch):
    if(stoch < 20 ):
        note_st = 10
    elif(20 <= stoch <80 ):
        note_st = 5
    else:
        note_st = 0
        
    return note_st

Fonction qui calcule la note technique

In [18]:
def calcultech(rsi, macd, sto):
    note_tech = 0.35*5 + 0.25*calcul_rsi(float(rsi)) + 0.25*calcul_macd(float(macd)) + 0.15 *calcul_stoch(float(sto))
    
    return note_tech

On calcule la note technique

In [19]:
note_tech = calcultech(df_data['RSI'][-1], df_data['MACD'][-1], df_data['SlowK'][-1])

In [20]:
note_tech

5.0

Fonction pour evaluer l'analyse financiere

In [21]:
def note_fin(liste):
    if liste[0] <= 10:
        res=0
    elif liste[0] >= 17:
        res=5
    else:
        res=10
    if liste[1] >= 9:
        res+=10
    elif liste[1] >= 7:
        res+=9
    elif liste[1] >= 5:
        res+=8
    elif liste[1] >= 3:
        res+=7
    elif liste[1] >= 2:
        res+=6
    elif liste[1] >= 1:
        res+=5
    else:
        res+=3
    if liste[2] >= 5:
        res+=10
    elif liste[2] >= 3:
        res+=8
    elif liste[2] >= 2:
        res+=7
    elif liste[2] >= 1:
        res+=5
    else:
        res+=3
    if liste[3] >= 4:
        res+=10
    elif liste[3] >= 2:
        res+=5
    else:
        res+=0
    return float(res/4)

On prends la note financiere

In [22]:
note_f = note_fin(finance) 

In [23]:
note_f 

3.75

Fonction qui gère l'information en temps réel

In [24]:
def getInfo(action):
    cpt=0
    filename=os.environ["PUBLIC"]+'\\'+action+'Info.csv'
    with open(filename, 'r') as textfile:
        for row in reversed(list(csv.DictReader(textfile, fieldnames = ("date", "info", "pertinence", "note"), delimiter = ';'))):
            date = row['date']
            if date == "date":
                break
            info = row['info']
            pertinence = row['pertinence']
            note = row['note']
            datetime.datetime.strptime(row['date'], "%Y-%m-%d %H:%M")
            limit = datetime.datetime.now()
            limit = limit - datetime.timedelta(hours=2)
            if datetime.datetime.strptime(row['date'], "%Y-%m-%d %H:%M") < limit:
                if cpt==0:
                    tab=[date,info,pertinence,note]
                    cpt=1
                else:
                    tab=tab,[date,info,pertinence,note]
    cpt = 0
    res=[]
    res.append(0)
    res.append('')
    for i in tab:
        res[0]+=5 + float(i[3]) * float(i[2])
        res[1]+=i[1]+'\n'
        cpt+=1
    res[0]=res[0]/cpt
    
    return res

In [25]:
resultat_info = getInfo(action)

ValueError: time data ' ' does not match format '%Y-%m-%d %H:%M'

In [26]:
note_info = resultat_info[0]
note_info

1.5

On genere le hazard

In [27]:
hazard = randint(1,8)
hazard

5

on calcule la note finale

In [28]:
note_finale = 0.23 * note_f + 0.20 * note_tech + 0.5 * note_info + 0.07 * hazard
note_finale

3.345

On emet une ecommandation en fonction de la note finale

In [29]:
if(note_finale < 3.33):
    recom = "VENDEZ !!!!"
elif( 3.33 <= note_finale < 7):
    recom = "NE FAITES RIEN! ATTENDEZ !!!"
else:
    recom = "ACHETEZ !!! "

On affiche la recommandation à l'utilisateur et la raison

In [30]:
print("\n \n")
print("Notre conseil :" + recom)
print("\n \n RAISON: "+ resultat_info[1])


 

Notre conseil :NE FAITES RIEN! ATTENDEZ !!!

 
 RAISON: la rupture des 20.8 se confirme
l'action passe sous le plancher a 20.26 risque de ne pas se relever

